In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import gc
import math
import random
import pickle
import pandas as pd
import numpy as np
import multiprocessing
from tqdm.auto import tqdm

from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, sampler

from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from imblearn.over_sampling import SMOTE, SMOTENC, SMOTEN, RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, CondensedNearestNeighbour, OneSidedSelection, NeighbourhoodCleaningRule

import shap
import catboost
from catboost import Pool, cv

pd.set_option('display.max_columns', 500)

In [2]:
train = pd.read_csv("./data/train.csv").drop(columns=['father', 'mother', 'gender'])
train.drop_duplicates(subset=train.columns.tolist()[5:20], inplace=True, ignore_index=True)
test = pd.read_csv("./data/test.csv").drop(columns=['father', 'mother', 'gender'])

train.iloc[:, 1:-1] = train.iloc[:, 1:-1].astype('category')
test.iloc[:, 1:] = test.iloc[:, 1:].astype('category')

answer = np.zeros(len(test)) - 1

train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id      248 non-null    object  
 1   trait   248 non-null    category
 2   SNP_01  248 non-null    category
 3   SNP_02  248 non-null    category
 4   SNP_03  248 non-null    category
 5   SNP_04  248 non-null    category
 6   SNP_05  248 non-null    category
 7   SNP_06  248 non-null    category
 8   SNP_07  248 non-null    category
 9   SNP_08  248 non-null    category
 10  SNP_09  248 non-null    category
 11  SNP_10  248 non-null    category
 12  SNP_11  248 non-null    category
 13  SNP_12  248 non-null    category
 14  SNP_13  248 non-null    category
 15  SNP_14  248 non-null    category
 16  SNP_15  248 non-null    category
 17  class   248 non-null    object  
dtypes: category(16), object(2)
memory usage: 9.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data colu

(None, None)

In [3]:
def catgbmc(inputX, inputY, validX, validY, params) :  
    var_categ = inputX.columns.tolist()
    model = CatBoostClassifier(
        cat_features=var_categ,
        **params,
        task_type='GPU',
        devices='0',
        )
  
    model.fit(
        inputX, inputY,
        eval_set=(inputX, inputY)
        );     

    pred = model.predict(validX)
    score = f1_score(validY, pred, average='macro')
    print(score)
    
    return model

In [4]:
X, y = train.drop(columns=['id', 'class']), (train['class'].values == 'A').astype('int')
X_test = test.drop(columns=['id']).copy()

params = {'iterations':100,
          'learning_rate':0.05,
          'l2_leaf_reg' : 10,
          'loss_function' : 'CrossEntropy',
          'eval_metric' : 'F1',
          'verbose':0,
          'random_seed':0}

modelA = catgbmc(X, y, X, y, params)
predA = modelA.predict(X_test)
answer += predA

1.0


In [5]:
train['class'].value_counts()

B    111
C     77
A     60
Name: class, dtype: int64

In [6]:
# text 형태의 categorical 변수들을 숫자형태로 변경

for i in tqdm(range(1, 15+1)) :
    target = str(i) if i >= 10 else "0"+str(i)
    try :   
        cols = sorted(train[f"SNP_{target}"].unique().tolist())  
        train[f"SNP_{target}"] = train[f"SNP_{target}"].map(lambda x : 0 if x==cols[0] else (1 if x==cols[1] else 2))
        test[f"SNP_{target}"] = test[f"SNP_{target}"].map(lambda x : 0 if x==cols[0] else (1 if x==cols[1] else 2))
    except :
        continue

train.info(), test.info()

  0%|          | 0/15 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id      248 non-null    object  
 1   trait   248 non-null    category
 2   SNP_01  248 non-null    category
 3   SNP_02  248 non-null    category
 4   SNP_03  248 non-null    category
 5   SNP_04  248 non-null    category
 6   SNP_05  248 non-null    category
 7   SNP_06  248 non-null    category
 8   SNP_07  248 non-null    category
 9   SNP_08  248 non-null    category
 10  SNP_09  248 non-null    category
 11  SNP_10  248 non-null    category
 12  SNP_11  248 non-null    category
 13  SNP_12  248 non-null    category
 14  SNP_13  248 non-null    category
 15  SNP_14  248 non-null    category
 16  SNP_15  248 non-null    category
 17  class   248 non-null    object  
dtypes: category(16), object(2)
memory usage: 9.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data colu

(None, None)

In [7]:
random_seed=0
strategy1 = {0 : 40, 1 : 70, 2 : 50}

under1 = RandomUnderSampler(sampling_strategy=strategy1, random_state=random_seed)
under2 = EditedNearestNeighbours()
under3 = RepeatedEditedNearestNeighbours()
under4 = AllKNN()
under5 = CondensedNearestNeighbour(random_state=random_seed)
under6 = OneSidedSelection(random_state=random_seed)
under7 = NeighbourhoodCleaningRule()

X, y = train.iloc[:, 1:-1], train['class'].map(lambda x : 0 if x=='A' else (1 if x=='B' else 2)).values

X1, y1 = under1.fit_resample(X, y)
X2, y2 = under2.fit_resample(X, y)
X3, y3 = under3.fit_resample(X, y)
X4, y4 = under4.fit_resample(X, y)
X5, y5 = under5.fit_resample(X, y)
X6, y6 = under6.fit_resample(X, y)
X7, y7 = under7.fit_resample(X, y)

In [8]:
random_seed=0
strategy2 = {0 : 100, 1 : 120, 2 : 110}

over1 = SMOTEN(sampling_strategy=strategy2, random_state=random_seed)
over2 = SMOTE(sampling_strategy=strategy2, random_state=random_seed)
over3 = RandomOverSampler(sampling_strategy=strategy2, random_state=random_seed)

X8, y8 = over1.fit_resample(X, y)
X9, y9 = over2.fit_resample(X, y)
X10, y10 = over3.fit_resample(X, y)

In [9]:
len(list(y1)+list(y2))

356

In [10]:
df_train = pd.concat([X1, X2, X3, X4, X5, X6, X7, X8, X9, X10], ignore_index=True)

df_train['class'] = list(y1)+list(y2)+list(y3)+list(y4)+list(y5)+list(y6)+list(y7)+list(y8)+list(y9)+list(y10)

df_train

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,1,1,2,0,2,0,2,2,0,2,0,2,1,2,1,1,0
1,1,1,2,0,1,1,1,1,0,1,0,2,1,2,1,0,0
2,1,0,2,0,1,0,2,2,0,1,1,1,1,2,1,2,0
3,1,0,2,0,2,0,2,2,0,1,1,2,1,2,1,2,0
4,1,0,2,0,1,1,2,1,0,0,0,2,2,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,2,2,0,0,0,1,1,1,2,0,1,1,0,2,0,1,2
2126,2,2,0,1,0,1,1,0,1,0,2,1,1,1,0,0,2
2127,2,1,1,1,0,0,1,0,1,0,1,0,1,2,0,0,2
2128,2,0,1,0,0,1,2,0,0,0,2,1,1,1,1,0,2


In [11]:
df_train.drop_duplicates(inplace=True, ignore_index=True)
df_train

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,1,1,2,0,2,0,2,2,0,2,0,2,1,2,1,1,0
1,1,1,2,0,1,1,1,1,0,1,0,2,1,2,1,0,0
2,1,0,2,0,1,0,2,2,0,1,1,1,1,2,1,2,0
3,1,0,2,0,2,0,2,2,0,1,1,2,1,2,1,2,0
4,1,0,2,0,1,1,2,1,0,0,0,2,2,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2,0,1,0,0,1,2,0,0,0,1,1,0,1,1,0,2
338,2,2,0,1,0,1,1,0,1,0,1,0,0,1,0,2,2
339,2,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,2
340,2,1,1,0,0,1,1,0,0,0,1,0,0,1,1,1,2


In [12]:
df_train['class'].value_counts()

1    124
2    123
0     95
Name: class, dtype: int64

In [13]:
encoder_len = 3

In [14]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super().__init__()
        self.encoding_dim = encoding_dim
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 24),
            nn.GELU(),
            nn.Linear(24, 8),
            nn.GELU(),
            nn.Linear(8, encoding_dim),
            nn.GELU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 8),
            nn.GELU(),
            nn.Linear(8, 24),
            nn.GELU(),
            nn.Linear(24, input_dim)
        )

    def forward(self, x):
        x1 = self.encoder(x)
        x2 = self.decoder(x1)
        return x1, x2

def ae_train(model, data_loader, criterion, optimizer, device, epochs=10):
    model.to(device)
    for epoch in range(epochs):
        epoch_loss = 0
        for x in data_loader:
            x = x[0].to(device)
            _, x_hat = model(x)
            loss = criterion(x_hat, x)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch {epoch}: loss = {epoch_loss / len(data_loader):.4f}')

In [15]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, input_dim*2),
            nn.ReLU(),
            nn.Linear(input_dim*2, input_dim),
            nn.ReLU(),
            nn.Linear(input_dim, input_dim//2),
            nn.ReLU()
            )
        self.fc_mu = nn.Linear(input_dim//2, latent_dim)
        self.fc_logvar = nn.Linear(input_dim//2, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, input_dim//2),
            nn.ReLU(),
            nn.Linear(input_dim//2, input_dim),
            nn.ReLU(),
            nn.Linear(input_dim, input_dim*2),
            nn.ReLU(),
            nn.Linear(input_dim*2, input_dim)
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def loss_fn(recon_x, x, mu, logvar):
    reconstruction_loss = F.mse_loss(recon_x, x, reduction='sum')
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return reconstruction_loss + kl_loss

def vae_train(model, optimizer, train_loader, device):
    model.train()
    train_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_fn(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        
        optimizer.step()
        
    return train_loss / len(train_loader.dataset)

def vae_test(model, test_loader, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_fn(recon_batch, data, mu, logvar).item()
    return test_loss / len(test_loader.dataset)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.Tensor(df_train.drop(columns=['class']).to_numpy())

# Create a dataset and data loader
dataset = torch.utils.data.TensorDataset(X)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=len(X), shuffle=True)

# Initialize the model, criterion, and optimizer
input_dim = X.shape[1]
encoding_dim = encoder_len
model = Autoencoder(input_dim, encoding_dim)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters())
epochs=3000

# Train the model
ae_train(model, data_loader, criterion, optimizer, device, epochs)

Epoch 0: loss = 1.4786
Epoch 1: loss = 1.4702
Epoch 2: loss = 1.4618
Epoch 3: loss = 1.4534
Epoch 4: loss = 1.4450
Epoch 5: loss = 1.4366
Epoch 6: loss = 1.4282
Epoch 7: loss = 1.4198
Epoch 8: loss = 1.4114
Epoch 9: loss = 1.4030
Epoch 10: loss = 1.3946
Epoch 11: loss = 1.3862
Epoch 12: loss = 1.3777
Epoch 13: loss = 1.3692
Epoch 14: loss = 1.3607
Epoch 15: loss = 1.3521
Epoch 16: loss = 1.3435
Epoch 17: loss = 1.3349
Epoch 18: loss = 1.3262
Epoch 19: loss = 1.3175
Epoch 20: loss = 1.3087
Epoch 21: loss = 1.2998
Epoch 22: loss = 1.2909
Epoch 23: loss = 1.2819
Epoch 24: loss = 1.2728
Epoch 25: loss = 1.2637
Epoch 26: loss = 1.2545
Epoch 27: loss = 1.2452
Epoch 28: loss = 1.2358
Epoch 29: loss = 1.2263
Epoch 30: loss = 1.2167
Epoch 31: loss = 1.2069
Epoch 32: loss = 1.1971
Epoch 33: loss = 1.1871
Epoch 34: loss = 1.1770
Epoch 35: loss = 1.1668
Epoch 36: loss = 1.1564
Epoch 37: loss = 1.1458
Epoch 38: loss = 1.1350
Epoch 39: loss = 1.1241
Epoch 40: loss = 1.1129
Epoch 41: loss = 1.1015
Ep

In [17]:
X1 = X.to(device)
X2 = torch.Tensor(test.drop(columns=['id']).to_numpy()).to(device)

pred_train = model(X1)
pred_test = model(X2)

# reconstruction error
loss_train = []
loss_test = []

for i in range(len(X1)) :
    loss = criterion(pred_train[1][i], X1[i]).detach().cpu().numpy()
    loss_train.append(loss)
for j in range(len(X2)) :
    loss = criterion(pred_test[1][j], X2[j]).detach().cpu().numpy()
    loss_test.append(loss)

# encoding values
enco_train = pred_train[0].detach().cpu().numpy()
enco_test = pred_test[0].detach().cpu().numpy()

trainLoss = pd.DataFrame(data=loss_train, columns=['ae_loss'])
testLoss = pd.DataFrame(data=loss_test, columns=['ae_loss'])

ae_train = pd.DataFrame(data=enco_train, columns=['ae_'+str(x) for x in range(enco_train.shape[1])])
ae_test = pd.DataFrame(data=enco_test, columns=['ae_'+str(x) for x in range(enco_train.shape[1])])

train2 = pd.concat([df_train, trainLoss, ae_train], axis=1)
test2 = pd.concat([test, testLoss, ae_test], axis=1)

train2.info(), test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   trait    342 non-null    category
 1   SNP_01   342 non-null    category
 2   SNP_02   342 non-null    category
 3   SNP_03   342 non-null    category
 4   SNP_04   342 non-null    category
 5   SNP_05   342 non-null    category
 6   SNP_06   342 non-null    category
 7   SNP_07   342 non-null    category
 8   SNP_08   342 non-null    category
 9   SNP_09   342 non-null    category
 10  SNP_10   342 non-null    category
 11  SNP_11   342 non-null    category
 12  SNP_12   342 non-null    category
 13  SNP_13   342 non-null    category
 14  SNP_14   342 non-null    category
 15  SNP_15   342 non-null    category
 16  class    342 non-null    int64   
 17  ae_loss  342 non-null    float32 
 18  ae_0     342 non-null    float32 
dtypes: category(16), float32(2), int64(1)
memory usage: 12.9 KB
<class 'pan

(None, None)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.Tensor(df_train.drop(columns=['class']).to_numpy())

input_dim = X.shape[1]
latent_dim = encoder_len
batch_size = len(X)
num_epochs = 1000

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data = X
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_data = X
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

model = VAE(input_dim, latent_dim).to(device)
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(num_epochs):
    train_loss = vae_train(model, optimizer, train_loader, device)
    test_loss = vae_test(model, test_loader, device)
    print(f"Epoch {epoch}: Train loss = {train_loss}, Test loss = {test_loss}")

Epoch 0: Train loss = 23.52336183068348, Test loss = 23.32525813230994
Epoch 1: Train loss = 23.33515225237573, Test loss = 23.15108307063231
Epoch 2: Train loss = 23.126090780336256, Test loss = 22.949398643092106
Epoch 3: Train loss = 22.95150167900219, Test loss = 22.747031763980264
Epoch 4: Train loss = 22.76657015259503, Test loss = 22.56300684164839
Epoch 5: Train loss = 22.55002598455775, Test loss = 22.37487721582602
Epoch 6: Train loss = 22.36127387152778, Test loss = 22.180662634777047
Epoch 7: Train loss = 22.17693056697734, Test loss = 21.972277903417396
Epoch 8: Train loss = 21.95991239491959, Test loss = 21.77980228892544
Epoch 9: Train loss = 21.771841305738302, Test loss = 21.558125456871345
Epoch 10: Train loss = 21.577295492964183, Test loss = 21.37647341008772
Epoch 11: Train loss = 21.365285773026315, Test loss = 21.148670218841374
Epoch 12: Train loss = 21.167576126187864, Test loss = 20.93522278188962
Epoch 13: Train loss = 20.94571083470395, Test loss = 20.719326

In [19]:
X1 = X.to(device)
X2 = torch.Tensor(test.drop(columns='id').to_numpy()).to(device)

pred_train = model(X1)
pred_test = model(X2)

# reconstruction error
loss_train = []
loss_test = []

for i in range(len(X1)) :
    loss = criterion(pred_train[0][i], X1[i]).detach().cpu().numpy()
    loss_train.append(loss)
for j in range(len(X2)) :
    loss = criterion(pred_test[0][j], X2[j]).detach().cpu().numpy()
    loss_test.append(loss)

# encoding values
enco_train = pred_train[1].detach().cpu().numpy()
enco_test = pred_test[1].detach().cpu().numpy()

trainLoss = pd.DataFrame(data=loss_train, columns=['vae_loss'])
testLoss = pd.DataFrame(data=loss_test, columns=['vae_loss'])

ae_train = pd.DataFrame(data=enco_train, columns=['vae_'+str(x) for x in range(enco_train.shape[1])])
ae_test = pd.DataFrame(data=enco_test, columns=['vae_'+str(x) for x in range(enco_train.shape[1])])

train3 = pd.concat([train2, trainLoss, ae_train], axis=1)
test3 = pd.concat([test2, testLoss, ae_test], axis=1)

train3.info(), test3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   trait     342 non-null    category
 1   SNP_01    342 non-null    category
 2   SNP_02    342 non-null    category
 3   SNP_03    342 non-null    category
 4   SNP_04    342 non-null    category
 5   SNP_05    342 non-null    category
 6   SNP_06    342 non-null    category
 7   SNP_07    342 non-null    category
 8   SNP_08    342 non-null    category
 9   SNP_09    342 non-null    category
 10  SNP_10    342 non-null    category
 11  SNP_11    342 non-null    category
 12  SNP_12    342 non-null    category
 13  SNP_13    342 non-null    category
 14  SNP_14    342 non-null    category
 15  SNP_15    342 non-null    category
 16  class     342 non-null    int64   
 17  ae_loss   342 non-null    float32 
 18  ae_0      342 non-null    float32 
 19  vae_loss  342 non-null    float32 
 20  vae_0     

(None, None)

In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train3.iloc[:, -(encoder_len+1)*2:] = scaler.fit_transform(train3.iloc[:, -(encoder_len+1)*2:])
test3.iloc[:, -(encoder_len+1)*2:] = scaler.transform(test3.iloc[:, -(encoder_len+1)*2:])

train3.describe()

,class,ae_loss,ae_0,vae_loss,vae_0
count,342.000000,3.420000e+02,3.420000e+02,3.420000e+02,3.420000e+02
mean,1.081871,-1.324003e-08,-2.456295e-09,1.319782e-08,-2.235038e-08
std,0.795345,1.001465e+00,1.001465e+00,1.001465e+00,1.001465e+00
min,0.000000,-1.532726e+00,-1.469719e+00,-2.012058e+00,-1.794122e+00
25%,0.000000,-8.221082e-01,-1.000921e+00,-7.364883e-01,-8.365247e-01
50%,1.000000,-2.216613e-01,-1.551384e-01,-1.481074e-01,-1.356279e-01
75%,2.000000,6.263688e-01,9.952123e-01,6.590430e-01,1.073616e+00
max,2.000000,3.490937e+00,1.594566e+00,4.236667e+00,1.955089e+00


In [21]:
test3.describe()

,ae_loss,ae_0,vae_loss,vae_0
count,175.000000,175.000000,175.000000,175.000000
mean,0.349333,0.142249,0.038692,-0.126360
std,1.276865,1.069330,0.991779,1.127576
min,-1.532726,-1.470191,-1.837792,-1.701904
25%,-0.508040,-1.033889,-0.725013,-1.110235
50%,0.175855,0.618940,-0.000575,-0.451472
75%,0.981910,1.102436,0.595854,1.134885
max,6.302737,1.611470,3.072975,1.955089


In [22]:
train3['class'].value_counts()

1    124
2    123
0     95
Name: class, dtype: int64

In [23]:
def catcv(inputX, inputY, params, cv_count) :  
    
    var_categ = inputX.columns.tolist()[:-(encoder_len+1)*2]
    
    cv_dataset = Pool(data=inputX,
                      label=inputY,
                      cat_features=var_categ)
    
    scores = cv(cv_dataset,
                params,
                fold_count=cv_count,
                stratified=True,
                plot=True)

In [24]:
params = {'iterations':100,
          'learning_rate':0.03,
          'loss_function' : 'CrossEntropy',
          'eval_metric' : 'F1',
          'verbose':0,
          'random_seed':2023}

In [25]:
X

tensor([[1., 1., 2.,  ..., 2., 1., 1.],
        [1., 1., 2.,  ..., 2., 1., 0.],
        [1., 0., 2.,  ..., 2., 1., 2.],
        ...,
        [2., 1., 0.,  ..., 1., 1., 0.],
        [2., 1., 1.,  ..., 1., 1., 1.],
        [2., 2., 1.,  ..., 1., 0., 1.]])

In [26]:
# B & notB 파생변수 없는 버전 성능 확인
X, y = train3.drop(columns=['class','trait']), (train3['class']==1).values.astype('int')
X_test = test3.drop(columns=['id','trait']).copy()

catcv(X, y, params, cv_count=5)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.96
bestIteration = 1

Training on fold [1/5]

bestTest = 0.96
bestIteration = 4

Training on fold [2/5]

bestTest = 0.84
bestIteration = 0

Training on fold [3/5]

bestTest = 0.9615384615
bestIteration = 1

Training on fold [4/5]

bestTest = 0.9230769231
bestIteration = 2



In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   SNP_01    342 non-null    category
 1   SNP_02    342 non-null    category
 2   SNP_03    342 non-null    category
 3   SNP_04    342 non-null    category
 4   SNP_05    342 non-null    category
 5   SNP_06    342 non-null    category
 6   SNP_07    342 non-null    category
 7   SNP_08    342 non-null    category
 8   SNP_09    342 non-null    category
 9   SNP_10    342 non-null    category
 10  SNP_11    342 non-null    category
 11  SNP_12    342 non-null    category
 12  SNP_13    342 non-null    category
 13  SNP_14    342 non-null    category
 14  SNP_15    342 non-null    category
 15  ae_loss   342 non-null    float64 
 16  ae_0      342 non-null    float64 
 17  vae_loss  342 non-null    float64 
 18  vae_0     342 non-null    float64 
dtypes: category(15), float64(4)
memory usage: 17.8 KB


In [42]:
def catgbmc(inputX, inputY, validX, validY, params) :  
    var_categ = inputX.columns.tolist()[:-(encoder_len+1)*2]
    model = CatBoostClassifier(
        cat_features=var_categ,
        **params,
        task_type='GPU',
        devices='0',
        )
  
    model.fit(
        inputX, inputY,
        eval_set=(inputX, inputY)
        );     

    pred = model.predict(validX)
    score = f1_score(validY, pred, average='macro')
    
    return score

In [43]:
from sklearn.model_selection import train_test_split

high = 0.9
for i in tqdm(range(2000)) :
    params = {'iterations':100,
          'learning_rate':0.3,
          'loss_function' : 'CrossEntropy',
          'eval_metric' : 'F1',
          'verbose':0,
          'random_seed':i}
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, random_state=i, shuffle=True)
    score = catgbmc(X_train, y_train, X_valid, y_valid, params)   
    
    if score >= high :
        high = score
        print(f"Random Seed : {i}, Score is {score}")

  0%|          | 0/2000 [00:00<?, ?it/s]

Random Seed : 1, Score is 0.9287901234567901
Random Seed : 2, Score is 0.9401162790697675
Random Seed : 5, Score is 0.9587339743589745
Random Seed : 7, Score is 0.9685496183206106
Random Seed : 17, Score is 0.9773327464788732
Random Seed : 19, Score is 0.9892540427751695
Random Seed : 327, Score is 0.9898271604938271


KeyboardInterrupt: 

In [30]:
# from sklearn.model_selection import train_test_split

# high = 0.9
# for i in tqdm(range(2000)) :
#     params = {'iterations':100,
#           'learning_rate':0.3,
#           'loss_function' : 'CrossEntropy',
#           'eval_metric' : 'F1',
#           'verbose':0,
#           'random_seed':i}
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, stratify=y, random_state=i, shuffle=True)
#     score = catgbmc(X_train, y_train, X_valid, y_valid, params)   
    
#     if score > high :
#         high = score
#         print(f"Random Seed : {i}, Score is {score}")

In [44]:
def catgbmc(inputX, inputY, validX, validY, params) :  
    var_categ = inputX.columns.tolist()[:-(encoder_len+1)*2]
    model = CatBoostClassifier(
        cat_features=var_categ,
        **params,
        task_type='GPU',
        devices='0',
        )
  
    model.fit(
        inputX, inputY,
        eval_set=(inputX, inputY)
        );     
    
    return model

In [58]:
from sklearn.model_selection import train_test_split

i = 327

params = {'iterations':100,
        'learning_rate':0.3,
        'loss_function' : 'CrossEntropy',
        'l2_leaf_reg' : 3,
        'eval_metric' : 'F1',
        'verbose':0,
        'random_seed':i}
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, random_state=i, shuffle=True)
model = catgbmc(X_train, y_train, X_valid, y_valid, params)   

pred1 = model.predict(X_train)
score1 = f1_score(y_train, pred1, average='macro')
pred2 = model.predict(X_valid)
score2 = f1_score(y_valid, pred2, average='macro')

score1, score2

(0.9862698914229906, 0.9898271604938271)

In [59]:
df = pd.DataFrame()
df['predA'] = answer
df['predB'] = model.predict_proba(X_test)[:,1]
df['pred'] = df['predA']
df.loc[df.predB >= 0.5, 'pred'] = 'B'
df.loc[df.predB < 0.5, 'pred'] = 'C'
df.loc[df.predA == 0, 'pred'] = 'A'
df

,predA,predB,pred
0,0.0,0.004194,A
1,-1.0,0.912491,B
2,-1.0,0.007454,C
3,-1.0,0.806894,B
4,0.0,0.003795,A
...,...,...,...
170,-1.0,0.982861,B
171,-1.0,0.005022,C
172,-1.0,0.008407,C
173,-1.0,0.953873,B


In [60]:
df.pred.value_counts()

B    86
A    51
C    38
Name: pred, dtype: int64

In [61]:
X

,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,ae_loss,ae_0,vae_loss,vae_0
0,1,2,0,2,0,2,2,0,2,0,2,1,2,1,1,-0.896724,-1.378629,-0.302016,1.732975
1,1,2,0,1,1,1,1,0,1,0,2,1,2,1,0,-0.172637,-0.973934,0.089400,1.046727
2,0,2,0,1,0,2,2,0,1,1,1,1,2,1,2,-0.934779,-1.300106,-1.023439,1.403667
3,0,2,0,2,0,2,2,0,1,1,2,1,2,1,2,-1.252937,-1.341154,-1.613754,1.652828
4,0,2,0,1,1,2,1,0,0,0,2,2,2,2,2,-0.500446,-1.071913,-0.677539,1.227719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,0,1,0,0,1,2,0,0,0,1,1,0,1,1,0,-0.331206,-0.816452,1.226426,0.442041
338,2,0,1,0,1,1,0,1,0,1,0,0,1,0,2,-0.391795,-0.410799,-0.525553,-0.061667
339,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,-0.071370,-0.652425,0.237524,0.290531
340,1,1,0,0,1,1,0,0,0,1,0,0,1,1,1,-1.448049,-0.701489,0.643033,0.273925


In [62]:
pd.DataFrame(data=model.get_feature_importance(), index=model.feature_names_, columns=['feature_importance']).sort_values('feature_importance', ascending=False)

,feature_importance
vae_0,27.810359
ae_0,19.597121
vae_loss,11.622371
ae_loss,9.022153
SNP_03,7.180539
SNP_15,5.625527
SNP_07,5.494928
SNP_11,5.321529
SNP_13,4.247737
SNP_08,2.944617


In [63]:
submit = pd.read_csv("submit_high1.csv")
submit['class'] = df['pred']
submit.to_csv("submit_last1.csv", index=False)
submit['class'].value_counts()

B    86
A    51
C    38
Name: class, dtype: int64